 <h1> <center> Manejo de Bases de Datos </center> </h1> 

<h2> Archivos </h2>

<p> De acuerdo a la documentación de <a href="https://pandas.pydata.org/pandas-docs/stable/reference/io.html"> Pandas</a> es posible importar distintos tipos de archivos, por ejemplo:</p>
<ul>
    <li> Archivos planos: Son archivos compuestos por caracteres sin formato, por lo general son los archivos .txt </li>
    <li> Archivos .csv o .tsv: Son archivos en los que las filas estan separadas por espacios tabulados, y las columans por comas </li>
</ul>
<br></br>
<b> Nota: </b> Muchas de las bases de datos que trabajaremos de aquí en adelantes se encuentran en los repositorios de <a href="http://mlr.cs.umass.edu/ml/"> UCI</a> y <a href="https://www.quandl.com"> QUANDL</a>, entre otros que en su oportunidad iremos comentando


In [1]:
import pandas as pd
import numpy as np
import openpyxl as oxl

In [ ]:
## En este caso importaremos un archivo Excel y un archivo csv
df   = pd.read_excel('mpg.xlsx','HojaDatos')
data = pd.read_csv('mpg.csv', sep = ";")
print(df.head(), "\n\n\n")
print(df.dtypes, "\n\n\n")
print(data.head(), "\n\n\n")
print(df.dtypes, "\n\n\n")
#dir(data)

Podemos transformar el tipo de campo que tiene nuestra base de datos, por ejemplo:

In [ ]:
df["horsepower"].astype(str)
print(df.dtypes, "\n\n\n")

In [ ]:
#df["horsepower"].astype(float)
df['horsepower'] = pd.to_numeric(df['horsepower'], errors='coerce')
print(df.dtypes, "\n\n\n")

<h2> Limpieza de bases de datos </h2>
<br></br>
<p>Por lo general, la limpieza se centra en trabajar con los datos vacíos, y es que en ocasiones las fuentes de información vienen incompletas por múltiples causas, por ejemplo: problemas de digitalización, problemas de ETL, entre otras razones.</p>

<p>En los siguientes casos vamos a tener una submuestra de la base <i>mpg</i> en la cual va a presentar los siguientes casos:</p>
<ul>
    <li> Valores pérdidos estándar </li>
    <li> Valores pérdidos no estándar </li>
    <li> Valores inesperados </li>
</ul>
    

In [ ]:
df   = pd.read_excel('mpg_clean.xlsx')
print(df)

In [ ]:
df.isnull()

In [ ]:
## Identificar los missing values
mv = ['na','--','n/a']
df   = pd.read_excel('mpg_clean.xlsx', na_values= mv)
print(df)


In [ ]:
# Número de valores pérdidos
df.isnull().sum()

In [ ]:
df.dtypes

<p> Existen distintas formas de reemplazar los valores pérdidos, podemos reemplazarlos con el valor de la media, mediana o moda, para datos númericos, moda para datos string o categóricos, o eliminar el registro </p>



In [ ]:
null1 = df['displacement'].mean()
null2 = df['horsepower'].median()
null3 = df['weight'].median()
null4 = df['model year'].mean()
null5 = df['acceleration'].median()
null6 = int(df['origin'].mode())

print('\n',null1,null2,null3,null4,null5,null6,'\n')

In [ ]:
df['displacement'].fillna(null1, inplace = True)
df['horsepower'].fillna(null2, inplace = True)
df['weight'].fillna(null3, inplace = True)
df['model year'].fillna(null4, inplace = True)
df['acceleration'].fillna(null5, inplace = True)
df['origin'].fillna(null6, inplace = True)
df.isnull().sum()

<p>Si revisamos, la variable <i>cylinders</i> presenta un error, y es que en la primera posición poseen caracteres alfabéticos, para ello debemos realizar una transformación adicional</p>

In [ ]:
df['cylinders'] = df['cylinders'].map(lambda x: x.lstrip('aAbBcCdvpft'))
df['cylinders']

<p> Si revisamos el campo de <i> car name </i> encontramos que hay registros numéricos que no son coherentes con el registro, por tanto, al ser la "llave", tenemos que eliminar el registro</p> 

In [ ]:
print(df['car name'])
df = df.drop([18, 24])
print(df)

In [ ]:
df['car name'].unique

<h2> Consultas </h2>
<p> Al poseer grandes fuentes de información, es necesario aplicar ciertas funciones sobre nuestras bases de datos que nos permitan visualizar:</p>
<ul>
    <li> Llave única</li>
    <li> Medidas de tendencia de un campo </li>
    <li> Total de un campo</li>
    <li> Ordenar la base de datos </li>
    <li> Eliminar duplicados </li>
    <li> Contar valores únicos </li>
</ul>

In [ ]:
print(df.dtypes, "\n\n\n")
df['cylinders'] = pd.to_numeric(df['cylinders'], errors='coerce')
print(df.dtypes, "\n\n\n")
print('\n SUMA \n\n',df.apply(np.sum),'\n\n')
print('\n MEDIA \n\n',df.iloc[:,0:8].apply(np.mean),'\n\n')
print('\n MEDIANA \n\n',df.iloc[:,0:8].apply(np.median),'\n\n')
print('\n MODA \n\n',df['car name'].mode(),'\n\n')

Tambien es posible ordenar la base de datos, y eliminar aquellos registros que esten duplicados

In [ ]:
# Ordenar la base de datos
print('\n ORDENAR POR AÑO \n\n',df.sort_values(by=['model year'], ascending=False),'\n\n')
print('\n ORDENAR POR AÑO Y PESO \n\n',df.sort_values(by=['model year','weight']),'\n\n')
print('\n ¿HAY DUPLICADOS? \n\n',df.duplicated(),'\n\n')
print('\n ELIMINAR DUPLICADOS \n\n',df.drop_duplicates(),'\n\n')
print('\n ELIMINAR DUPLICADOS POR COLUMNAS \n\n',df.drop_duplicates(['mpg']),'\n\n')
df = df.drop_duplicates()

In [ ]:
print(df)

También podemos contar valores únicos, de la siguiente manera:

In [ ]:
print('\n\n CONTAR VALORES ÚNICOS \n\n',df.nunique(),'\n\n')
print('\n\n CONTAR VALORES ÚNICOS \n\n',df.count(),'\n\n')

<h2> Combinar, unir y concatenar </h2>
<br></br>
<p> En el momento de manejar distintas bases de datos, es importante conocer como podemos obtener información de cada una de ellas</p>
<p> En esta sección veremos como combinar, unir y concatenar bases de datos </p>
<ul>
    <li> Para concatenar un marco de datos, utilizamos la función .concat(), esta función concatena un marco de datos y devuelve un nuevo marco de datos</li>
    <li> Para concatenar un marco de datos, utilizamos la función .append() esta función concatena a lo largo del eje = 0, es decir, el índice. </li>
    <li> Para juntar bases de datos utilizamos la función .merge()</li>
 </ul>

<p> Para estos ejercicios tomaremos tres tipos de bases diferentes que nos permitan combinar, unir y juntar. También ubicaremos la llave </p>

In [2]:
## Bases de datos
df = pd.read_excel('mpg.xlsx','HojaDatos')
print(df.shape)
# Separar en subconjuntos de índices
df1 = df.iloc[0:199]
df2 = df.iloc[200:298]
# Separar en columnas
df3 = df.iloc[0:10,[0,1,2,3,4,8]]
df4 = df.iloc[0:13,[5,6,7,8]]
print('\n\n CONTAR VALORES ÚNICOS \n\n',df.nunique(),'\n\n')
print('\n\n CONTAR VALORES ÚNICOS \n\n',df.count(),'\n\n')

(301, 9)


 CONTAR VALORES ÚNICOS 

 mpg             109
cylinders         5
displacement     77
horsepower       87
weight          275
acceleration     90
model year       13
origin            3
car name        301
dtype: int64 




 CONTAR VALORES ÚNICOS 

 mpg             301
cylinders       301
displacement    301
horsepower      301
weight          301
acceleration    301
model year      301
origin          301
car name        301
dtype: int64 




In [3]:
# Concatenación
df5 = pd.concat([df1,df2]) # , ignore_index=True
df6 = pd.concat([df1,df2],keys=['train', 'test'])
print(df5.shape,'\n\n')
print(df6,'\n\n')

df5 = df1.append(df2)
print(df5.shape)


(300, 9) 


            mpg  cylinders  displacement  horsepower  weight  acceleration  \
train 0    18.0          8         307.0       130.0    3504          12.0   
      1    15.0          8         350.0       165.0    3693          11.5   
      2    18.0          8         318.0       150.0    3436          11.0   
      3    16.0          8         304.0       150.0    3433          12.0   
      4    17.0          8         302.0       140.0    3449          10.5   
...         ...        ...           ...         ...     ...           ...   
test  296  27.0          4         140.0        86.0    2790          15.6   
      297  44.0          4          97.0        52.0    2130          24.6   
      298  32.0          4         135.0        84.0    2295          11.6   
      299  28.0          4         120.0        79.0    2625          18.6   
      300  31.0          4         119.0        82.0    2720          19.4   

           model year  origin                     c

<h3> Uniones </h3>
<p> Las uniones consisten en combinaciones de dos bases de datos con el uso de una llave, que por buena práctica, en al menos una de las bases debe ser única. </p> 
<p> Con este tipo de acciones podemos agregar información, buscar información relevante y estructurar los datos </p>
<br></br>
<img src= 'merge.png',width=400,heigth=1000, alt="Flowers in Chania">
<center> <i> Imagen tomada de <a href="https://www.geeksforgeeks.org/python-pandas-merging-joining-and-concatenating/"> geeksforgeeks</a> </i></center>

In [ ]:
union1 = pd.merge(df3, df4, on='car name')
union1

In [ ]:
union2 = pd.merge(df3, df4, how='left', on=['car name'])
union3 = pd.merge(df3, df4, how='right',  on=['car name'])
union4 = pd.merge(df3, df4, how='inner',  on=['car name'])
union5 = pd.merge(df3, df4, how='outer',  on=['car name'])

<h3> Exportar base </h3>
<p>Al trabajar con bases de datos, en muchas ocasiones necesitamos exportar los resultados para utilizarla en otros entornos, por espacio, por eficiencia, o por otros criterios.</p>
<p> En este caso exportaremos la base de datos en 3 escenarios: </p>
<ul>
    <li> .txt </li>
    <li> .csv </li>
    <li> .xlsx </li>
</ul>

In [ ]:
df.to_excel("mpg1.xlsx", "Sheet1")
df.to_csv("mpg1.csv")
df.to_csv("mpg1.txt")
